In [1]:
import pygame
import random
import sys

#
# Simple scoring logic:
# If kicker_side == goalie_side => 30% success,
# otherwise => 80% success.
#
def get_score_probability(kicker_side, goalie_side):
    if kicker_side == goalie_side:
        return 0.3
    else:
        return 0.8

def main():
    pygame.init()
    WIDTH, HEIGHT = 800, 600
    screen = pygame.display.set_mode((WIDTH, HEIGHT))
    pygame.display.set_caption("Penalty Kick Game with Animation")

    # Colors
    GREEN = (34, 139, 34)
    WHITE = (255, 255, 255)
    BLACK = (0, 0, 0)
    RED   = (220, 20, 60)
    BLUE  = (70, 130, 180)
    DARK_GRAY = (50, 50, 50)
    LIGHT_GRAY = (170, 170, 170)
    GRAY  = (100, 100, 100)

    # Fonts
    font_small  = pygame.font.SysFont(None, 24)
    font_medium = pygame.font.SysFont(None, 32)
    font_large  = pygame.font.SysFont(None, 48)

    # Layout
    top_bar_height = 160
    goal_line_y = top_bar_height + 40

    # Probability that goalie dives left (slider-controlled)
    p = 0.5

    # Attempts
    total_attempts = 5
    attempt_count = 1
    goals_scored = 0

    # Stats
    shots_left = 0
    shots_right = 0
    goalie_dives_left_count = 0
    goalie_dives_right_count = 0

    # Goalie geometry (start in center)
    goalie_w, goalie_h = 60, 60
    goalie_center_x = (WIDTH - goalie_w) // 2
    goalie_center_y = goal_line_y - goalie_h // 2

    # Ball geometry (start at bottom center)
    ball_radius = 20
    ball_start_x = WIDTH // 2
    ball_start_y = HEIGHT - 100

    # We'll store the current positions that get updated each frame
    goalie_x = goalie_center_x
    goalie_y = goalie_center_y
    ball_x = ball_start_x
    ball_y = ball_start_y

    # State machine
    # 1) WAIT_FOR_INPUT
    # 2) SHOOTING (animate ball + goalie)
    # 3) SHOW_RESULT (display GOAL/MISS)
    # 4) GAME_OVER
    state = "WAIT_FOR_INPUT"
    result_text = ""
    result_time = 0
    result_duration = 2000  # 2 seconds for SHOW_RESULT

    # For SHOOTING animation
    shot_time = 0
    shot_duration = 1000  # 1 second (in ms)

    # We'll store the side chosen by kicker (0=Left, 1=Right)
    # and the side chosen by goalie (0=Left, 1=Right).
    kicker_side = None
    goalie_side = None

    # We'll store start and end coordinates for ball & goalie
    ball_start_pos = (ball_start_x, ball_start_y)
    ball_end_pos   = (ball_start_x, goal_line_y + 20)  # straight up by default

    goalie_start_pos = (goalie_center_x, goalie_center_y)
    goalie_end_pos   = (goalie_center_x, goalie_center_y)  # stays center by default

    # Buttons
    btn_left_rect  = pygame.Rect(50, HEIGHT - 80, 150, 50)
    btn_right_rect = pygame.Rect(WIDTH - 200, HEIGHT - 80, 150, 50)
    btn_restart_rect = pygame.Rect(WIDTH - 140, 110, 120, 40)

    # Slider geometry for p
    slider_x = 520
    slider_y = 80
    slider_width = 200
    slider_height = 5
    handle_radius = 10
    dragging_slider = False

    clock = pygame.time.Clock()

    def slider_rect():
        return pygame.Rect(slider_x, slider_y - handle_radius, slider_width, handle_radius*2)

    def reset_game():
        nonlocal attempt_count, goals_scored
        nonlocal shots_left, shots_right
        nonlocal goalie_dives_left_count, goalie_dives_right_count
        nonlocal state, result_text, result_time
        nonlocal shot_time, kicker_side, goalie_side

        attempt_count = 1
        goals_scored = 0
        shots_left = 0
        shots_right = 0
        goalie_dives_left_count = 0
        goalie_dives_right_count = 0

        state = "WAIT_FOR_INPUT"
        result_text = ""
        result_time = 0
        shot_time = 0
        kicker_side = None
        goalie_side = None

    # A small helper to linearly interpolate from a to b by fraction t in [0..1]
    def lerp(a, b, t):
        return a + (b - a) * t

    running = True
    while running:
        dt = clock.tick(60)  # up to 60 FPS
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False

            # MOUSE DOWN
            if event.type == pygame.MOUSEBUTTONDOWN and event.button == 1:
                mx, my = event.pos
                # Check if clicked "Restart"
                if btn_restart_rect.collidepoint(mx, my):
                    reset_game()

                # Check if clicked slider
                if slider_rect().collidepoint(mx, my):
                    dragging_slider = True
                    # Set p based on slider
                    p = (mx - slider_x) / slider_width
                    p = max(0.0, min(1.0, p))

                if state == "WAIT_FOR_INPUT":
                    # Kick LEFT
                    if btn_left_rect.collidepoint(mx, my):
                        shots_left += 1
                        kicker_side = 0  # left
                        # Goalie side
                        goalie_side = 0 if (random.random() < p) else 1
                        if goalie_side == 0:
                            goalie_dives_left_count += 1
                        else:
                            goalie_dives_right_count += 1

                        # Prepare SHOOTING animation
                        shot_time = 0
                        # Ball start
                        ball_start_pos = (ball_start_x, ball_start_y)
                        # If kicker_side=0 (left), aim ball a bit left near goal line
                        if kicker_side == 0:
                            ball_end_pos = (ball_start_x - 100, goal_line_y + 20)
                        else:
                            ball_end_pos = (ball_start_x + 100, goal_line_y + 20)

                        # Goalie start
                        goalie_start_pos = (goalie_center_x, goalie_center_y)
                        # If goalie_side=0 (left), dive left
                        if goalie_side == 0:
                            goalie_end_pos = (goalie_center_x - 100, goalie_center_y)
                        else:
                            goalie_end_pos = (goalie_center_x + 100, goalie_center_y)

                        state = "SHOOTING"

                    # Kick RIGHT
                    elif btn_right_rect.collidepoint(mx, my):
                        shots_right += 1
                        kicker_side = 1  # right
                        # Goalie side
                        goalie_side = 0 if (random.random() < p) else 1
                        if goalie_side == 0:
                            goalie_dives_left_count += 1
                        else:
                            goalie_dives_right_count += 1

                        # Prepare SHOOTING animation
                        shot_time = 0
                        ball_start_pos = (ball_start_x, ball_start_y)
                        if kicker_side == 0:
                            ball_end_pos = (ball_start_x - 100, goal_line_y + 20)
                        else:
                            ball_end_pos = (ball_start_x + 100, goal_line_y + 20)

                        goalie_start_pos = (goalie_center_x, goalie_center_y)
                        if goalie_side == 0:
                            goalie_end_pos = (goalie_center_x - 100, goalie_center_y)
                        else:
                            goalie_end_pos = (goalie_center_x + 100, goalie_center_y)

                        state = "SHOOTING"

            # MOUSE UP
            if event.type == pygame.MOUSEBUTTONUP and event.button == 1:
                dragging_slider = False

            # MOUSE MOTION
            if event.type == pygame.MOUSEMOTION and dragging_slider:
                mx, my = event.pos
                p = (mx - slider_x) / slider_width
                p = max(0.0, min(1.0, p))

            # If in GAME_OVER, allow ESC to quit
            if state == "GAME_OVER":
                if event.type == pygame.KEYDOWN and event.key == pygame.K_ESCAPE:
                    running = False

        # -- Update Logic per State --
        if state == "SHOOTING":
            shot_time += dt
            # fraction of the animation completed [0..1]
            frac = min(1.0, shot_time / shot_duration)

            # Lerp ball & goalie
            ball_x = lerp(ball_start_pos[0], ball_end_pos[0], frac)
            ball_y = lerp(ball_start_pos[1], ball_end_pos[1], frac)

            goalie_x = lerp(goalie_start_pos[0], goalie_end_pos[0], frac)
            goalie_y = lerp(goalie_start_pos[1], goalie_end_pos[1], frac)

            if frac >= 1.0:
                # Animation finished
                # Check GOAL or MISS
                score_prob = get_score_probability(kicker_side, goalie_side)
                shot_success = (random.random() < score_prob)
                result_text = "GOAL!" if shot_success else "MISS!"
                if shot_success:
                    goals_scored += 1

                # Move to SHOW_RESULT
                state = "SHOW_RESULT"
                result_time = 0

        elif state == "SHOW_RESULT":
            result_time += dt
            if result_time >= result_duration:
                # Next attempt if any left
                if attempt_count < total_attempts:
                    attempt_count += 1
                    state = "WAIT_FOR_INPUT"
                    # Reset positions to default
                    ball_x = ball_start_x
                    ball_y = ball_start_y
                    goalie_x = goalie_center_x
                    goalie_y = goalie_center_y
                else:
                    state = "GAME_OVER"

        # --- DRAW ---
        screen.fill(GREEN)

        # Draw top bar
        top_bar_rect = pygame.Rect(0, 0, WIDTH, top_bar_height)
        pygame.draw.rect(screen, DARK_GRAY, top_bar_rect)

        # Draw field line
        pygame.draw.line(screen, WHITE, (0, goal_line_y), (WIDTH, goal_line_y), 4)

        # Draw goalie
        pygame.draw.rect(screen, BLUE, (goalie_x, goalie_y, goalie_w, goalie_h))

        # Draw ball
        pygame.draw.circle(screen, WHITE, (int(ball_x), int(ball_y)), ball_radius)

        # Attempt & stats
        displayed_attempt = min(attempt_count, total_attempts)
        shots_completed = max(0, displayed_attempt - 1)
        success_rate = (goals_scored / shots_completed * 100) if shots_completed > 0 else 0.0

        stats_line1 = f"Attempt: {displayed_attempt}/{total_attempts}   Goals: {goals_scored}   Rate: {success_rate:.1f}%"
        txt1_surf = font_medium.render(stats_line1, True, WHITE)
        screen.blit(txt1_surf, (20, 10))

        stats_line2 = (f"Shots(L)={shots_left},(R)={shots_right} | "
                       f"Goalie(L)={goalie_dives_left_count},(R)={goalie_dives_right_count}")
        txt2_surf = font_small.render(stats_line2, True, WHITE)
        screen.blit(txt2_surf, (20, 50))

        # Slider label
        prob_label_surf = font_small.render("Goalie Dive Probability", True, WHITE)
        prob_label_rect = prob_label_surf.get_rect(center=(slider_x + slider_width/2, 30))
        screen.blit(prob_label_surf, prob_label_rect)

        # Slider bar
        pygame.draw.rect(screen, GRAY, (slider_x, slider_y, slider_width, slider_height))
        # Slider handle
        handle_x = slider_x + int(p * slider_width)
        handle_y = slider_y + slider_height // 2
        pygame.draw.circle(screen, LIGHT_GRAY, (handle_x, handle_y), handle_radius)
        # p value
        p_value_surf = font_small.render(f"{p:.2f}", True, WHITE)
        p_value_rect = p_value_surf.get_rect(center=(slider_x + slider_width/2, slider_y + 35))
        screen.blit(p_value_surf, p_value_rect)

        # Restart button
        pygame.draw.rect(screen, LIGHT_GRAY, btn_restart_rect)
        restart_txt = font_medium.render("Restart", True, BLACK)
        restart_rect = restart_txt.get_rect(center=btn_restart_rect.center)
        screen.blit(restart_txt, restart_rect)

        # If in SHOW_RESULT, overlay text
        if state == "SHOW_RESULT":
            color = RED if "MISS" in result_text else WHITE
            result_surf = font_large.render(result_text, True, color)
            result_rect = result_surf.get_rect(center=(WIDTH // 2, HEIGHT // 2))
            screen.blit(result_surf, result_rect)

        # If GAME_OVER, show final message
        if state == "GAME_OVER":
            final_rate = (goals_scored / float(total_attempts)) * 100 if total_attempts else 0
            game_over_text = f"Game Over! You scored {goals_scored}/{total_attempts}  ({final_rate:.1f}%)."
            go_surf = font_large.render(game_over_text, True, BLACK)
            go_rect = go_surf.get_rect(center=(WIDTH // 2, HEIGHT // 2))
            screen.blit(go_surf, go_rect)

            exit_text = "Press ESC or close window to exit."
            exit_surf = font_medium.render(exit_text, True, BLACK)
            exit_rect = exit_surf.get_rect(center=(WIDTH // 2, HEIGHT // 2 + 50))
            screen.blit(exit_surf, exit_rect)

        # Bottom buttons: Kick LEFT, Kick RIGHT
        pygame.draw.rect(screen, LIGHT_GRAY, btn_left_rect)
        left_txt = font_medium.render("Kick LEFT", True, BLACK)
        left_txt_rect = left_txt.get_rect(center=btn_left_rect.center)
        screen.blit(left_txt, left_txt_rect)

        pygame.draw.rect(screen, LIGHT_GRAY, btn_right_rect)
        right_txt = font_medium.render("Kick RIGHT", True, BLACK)
        right_txt_rect = right_txt.get_rect(center=btn_right_rect.center)
        screen.blit(right_txt, right_txt_rect)

        pygame.display.flip()

    pygame.quit()
    sys.exit()

if __name__ == "__main__":
    main()


pygame 2.6.1 (SDL 2.28.4, Python 3.13.1)
Hello from the pygame community. https://www.pygame.org/contribute.html


2025-02-19 11:28:38.598 Python[93741:13528154] WARNING: Secure coding is automatically enabled for restorable state! However, not on all supported macOS versions of this application. Opt-in to secure coding explicitly by implementing NSApplicationDelegate.applicationSupportsSecureRestorableState:.


SystemExit: 

/Users/ryan/Library/Python/3.13/lib/python/site-packages/IPython/core/interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
